In [1]:
#imports
import geopandas as gpd
import pandas as pd
import rasterio
import rasterio.mask
import tempfile
import fiona
import os
import glob
import numpy as np
import traceback
import subprocess
import json

In [2]:
# get the working directory
cwd = os.getcwd()

# warning messages
message = {}

# get the shapefile folder
shapefile_folder = os.path.join(cwd, 'shapefile')
# Pattern to match shapefile (shapefile typically have extensions like .shp)
shapefile_pattern = os.path.join(shapefile_folder, '*.shp')

shapefiles_list = glob.glob(shapefile_pattern)
# print(shapefiles_list)

# Count the number of shapefile
num_shapefiles = len(shapefiles_list)
# print(num_shapefiles)

# get the java folder
java_folder = os.path.join(cwd, 'source/java')

# get the csv folder
csv_folder = os.path.join(cwd, 'source/csv_files')
# Ensure the output directory exists and remove all existing CSV files in the output folder
if os.path.exists(csv_folder):
    for file in glob.glob(os.path.join(csv_folder, '*.csv')):
        os.remove(file)
    else:
        os.makedirs(csv_folder, exist_ok=True)
# get the modified csv folder path
modified_csv_folder = os.path.join(cwd, 'source/modified_csv_for_curr_QMD')
if os.path.exists(modified_csv_folder):
        for file in glob.glob(os.path.join(modified_csv_folder, '*.csv')):
            os.remove(file)
        else:
            os.makedirs(modified_csv_folder, exist_ok=True)
result_folder = os.path.join(cwd, 'result')
if os.path.exists(result_folder):
        for file in glob.glob(os.path.join(result_folder, '*.csv')):
            os.remove(file)
        else:
            os.makedirs(result_folder, exist_ok=True)
# get the raster file
raster_folder = os.path.join(cwd,'source/rasters')
tiffile = os.path.join(raster_folder,'kl_comp.tif')
# Get the parent directory
parent_dir = os.path.dirname(cwd)



In [3]:
def filter_standID_for_generatingCSV_for_CurrQMD(lis):
    dic_for_standID_with_currQMD = {}
    for item in lis:
        if 'curr_QMD' in item and 'curr_TPA' in item:
            try:
                # Convert the values to floats
                curr_qmd = float(item['curr_QMD'][0])
                curr_tpa = float(item['curr_TPA'][0])
                # Check if the values are not NaN and not zero
                if not np.isnan(curr_qmd) and not np.isnan(curr_tpa) and curr_qmd != 0 and curr_tpa != 0:
                    stand_id = int(item['StandID'][0])
                    dic_for_standID_with_currQMD[stand_id] = [int(item['curr_QMD'][0]),int(item['curr_TPA'][0])]
            except (ValueError, TypeError):
                print("absence due to invalid values")
    return dic_for_standID_with_currQMD

        
# dic = filter_standID_for_generatingCSV_for_CurrQMD(fields_from_shapefile)
# print(dic)

In [4]:
# map the numbers to characters
def number_to_letters_fast(nums):
    results = []
    # Iterate over the numbers
    for n in nums:
        letter = ''
        while n > 0:
            n -= 1  
            letter = chr(n % 26 + 97) + letter  
            n //= 26  
        results.append(letter)  
    return results

In [5]:
def to_csv(out_image, csv_folder, index, parameters, csv_files, stand_id, dict_for_standIDnCrrnQMD_from_allFields, csv_with_modified_qmd):
        """
        Process the raster file and save the band data to CSV files for each shape.

        Parameters:
        - out_image (numpy array): Masked raster data.
        - model_parameters (dict): Dictionary containing shape parameters.
        - output_folder (str): Folder to save the CSV files.
        Returns:
        - List of CSV filenames.
        """     
        os.makedirs(csv_folder, exist_ok=True)
        column_names = ['CMI_wt','dem30_usa','Subz_Name','M_PPT01','PRATIO','B_SHM','USA_Slope','M_Tmin03','TOTAL_LITH','IFC_Solar']


        with rasterio.open(tiffile) as src:
            # Flatten band data and create DataFrame
            band_data = {f'band_{j+1}': out_image[j].flatten() for j in range(out_image.shape[0])}
            df = pd.DataFrame(band_data)
            # Replace the nodata values with NaN
            for band in band_data:
                df[band] = df[band].replace(src.nodata, np.nan)
            df_cleaned = df    
            if not df_cleaned.empty:
                if len(column_names) >= df_cleaned.shape[1]:
                    df_cleaned.columns = column_names[:df_cleaned.shape[1]]
                # Extend the csv file with the model parameters
                param_df = pd.DataFrame({key: [value[0]] * len(df_cleaned) for key, value in parameters.items()})
                df_extended = pd.concat([df_cleaned, param_df], axis=1)
                # drop the NaN values
                df_extended = df_extended.dropna()
                cols = ['Subz_Name', 'TOTAL_LITH']
                df_extended[cols] = df_extended[cols].astype('int')
                for col in cols:
                    df_extended[col] = number_to_letters_fast(df_extended[col].values)
                # Save to CSV
                csv_filename = f"csv_for_stand_{stand_id}.csv"
                csv_path = os.path.join(csv_folder, csv_filename)
                csv_files.append(csv_path)
                df_extended.to_csv(csv_path, index=False)
                if stand_id in dict_for_standIDnCrrnQMD_from_allFields:
                     Current_QMD =  dict_for_standIDnCrrnQMD_from_allFields[stand_id]
                     print(Current_QMD)
                     df_extended['QMD'] = Current_QMD[0]
                     new_file_name = f"with_cur_QMD_{csv_filename}"
                     csv_with_modified_qmd.append(os.path.join(modified_csv_folder, new_file_name))
                     df_extended.to_csv(os.path.join(modified_csv_folder, new_file_name), index=False)

  

In [6]:

# csvfile list
csv_files = []
# Define desired columns
desired_columns = ['QMD','PM_PROP','TO_PROP','DF_PROP','SP_PROP','OH_PROP', 'OC_PROP','WH_PROP','GF_PROP','WF_PROP']

stand_id = []
# --------------------
stand_id = []
fields_from_shapefile = []
dict_for_standIDnCrrnQMD_from_allFields = {}
csv_with_current_qmd_and_tpa = []
# ---------------------

if os.path.exists(csv_folder):
    for file in glob.glob(os.path.join(csv_folder, '*.csv')):
        os.remove(file)
else:
    os.makedirs(csv_folder, exist_ok=True)

for shapefile in shapefiles_list:
    # Path to shapefile
    shapefile_path = shapefile
    # Read the main shapefile
    try:
        ply = gpd.read_file(shapefile_path)
    except Exception as ex:
        print(f"Unable to read shapefile  {shapefile_path}. Your file may be corrupt: {ex}")
        raise

    # Change CRS to match raster
    ply = ply.to_crs(epsg=3857)
    with tempfile.NamedTemporaryFile() as tf:
        crsName = tf.name
    ply.to_file(crsName)  # Write new shapefile 
    # Read and process shapefile
    with rasterio.open(tiffile) as geotiff:
        with fiona.open(crsName, "r") as shapefile:
            shapes = [feature["geometry"] for feature in shapefile]
            for index, shape in enumerate(shapes):
                try:
                    out_image, out_transform = rasterio.mask.mask(geotiff, [shape], crop=True, all_touched=True)                 
                    # Extract desired columns from the clipped shapefile
                    clipped_shapefile = ply.clip(gpd.GeoDataFrame(geometry=[shape], crs=ply.crs))
                    fields_from_shapefile.append(clipped_shapefile.copy().to_dict(orient='list'))
                    dict_for_standIDnCrrnQMD_from_allFields = filter_standID_for_generatingCSV_for_CurrQMD(fields_from_shapefile)
                    extracted_data = clipped_shapefile[desired_columns].copy()
                    desired_columns = ['QMD','PM_PROP','TO_PROP','DF_PROP','SP_PROP','OH_PROP', 'OC_PROP','WH_PROP','GF_PROP','WF_PROP']
                    extracted_data["W_ShadeTol"] = extracted_data["PM_PROP"] * 1.17 + extracted_data["TO_PROP"] * 3.67 + extracted_data["DF_PROP"] * 2.78 + extracted_data["SP_PROP"] * 2.66 + extracted_data["OH_PROP"] * 3 + extracted_data["OC_PROP"] * 2.5 + extracted_data["WH_PROP"] * 4.96+ extracted_data["GF_PROP"] * 4.01 + extracted_data["WF_PROP"] * 4.33
                    extracted_data["W_DroughtTol"] = extracted_data["PM_PROP"] * 4.97 + extracted_data["TO_PROP"] * 4 + extracted_data["DF_PROP"] * 2.62 + extracted_data["SP_PROP"] * 2.67 + extracted_data["OH_PROP"] * 3 + extracted_data["OC_PROP"] * 2.5 + extracted_data["WH_PROP"] * 1.17 + extracted_data["GF_PROP"] * 2.33 + extracted_data["WF_PROP"] * 1.91                 
                    # Extract StandID as integer and append to stand_id list -- we are getting this stand id from shapefile 
                    stand_id_value = int(clipped_shapefile['StandID'].values[0])
                    stand_id.append(stand_id_value)                  
                    # Convert the extracted data to a dictionary
                    extracted_data_dict_list = extracted_data.to_dict(orient='records')
                    # convert the list of dictionaries into a single dictionary if needed
                    extracted_data_dict = extracted_data.to_dict(orient='list')
                    # Call to_csv function to create and save the CSV file
                    to_csv(out_image, csv_folder, index, extracted_data_dict, csv_files, stand_id[-1], dict_for_standIDnCrrnQMD_from_allFields, csv_with_current_qmd_and_tpa)
                except Exception as ex:
                    print(f"Error processing shape {index}: {str(ex)}")
                    traceback.print_exc()  # Print the full traceback
                    print("\n")
                    print("\n")
                    print("\n")



[8, 700]
[11, 400]
[12, 390]


In [7]:
print(fields_from_shapefile)

[{'Id': [0], 'StandID': [101.0], 'QMD': [10.0], 'PM_PROP': [0.0], 'TO_PROP': [0.0], 'DF_PROP': [1.0], 'SP_PROP': [0.0], 'OH_PROP': [0.0], 'OC_PROP': [0.0], 'WH_PROP': [0.0], 'GF_PROP': [0.0], 'WF_PROP': [0.0], 'curr_QMD': [8.0], 'curr_TPA': [700.0], 'geometry': [<POLYGON ((-13703980.358 5306951.356, -13723331.137 5306951.356, -13714865.1...>]}, {'Id': [0], 'StandID': [102.0], 'QMD': [10.0], 'PM_PROP': [0.0], 'TO_PROP': [0.0], 'DF_PROP': [1.0], 'SP_PROP': [0.0], 'OH_PROP': [0.0], 'OC_PROP': [0.0], 'WH_PROP': [0.0], 'GF_PROP': [0.0], 'WF_PROP': [0.0], 'curr_QMD': [11.0], 'curr_TPA': [400.0], 'geometry': [<POLYGON ((-13699948.89 5198320.633, -13711161.985 5196218.178, -13710110.75...>]}, {'Id': [0], 'StandID': [103.0], 'QMD': [10.0], 'PM_PROP': [0.0], 'TO_PROP': [0.0], 'DF_PROP': [0.0], 'SP_PROP': [0.0], 'OH_PROP': [0.2], 'OC_PROP': [0.8], 'WH_PROP': [0.0], 'GF_PROP': [0.0], 'WF_PROP': [0.0], 'curr_QMD': [12.0], 'curr_TPA': [390.0], 'geometry': [<POLYGON ((-13653344.465 5053251.22, -13652

In [8]:
# this code will filter out the csv that are not empty and can be fed to model.

def filter_csv(folder_path):
    # List all CSV files in the folder
    csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]
    if csv_files is None:
        return 
    # Initialize a list to store the 'stand' values from non-empty CSV files
    stand_id_result = []
    csv_for_result = []
    # Check each CSV file
    for files in csv_files:
        try:
            df = pd.read_csv(files)  # Read the CSV file into a DataFrame
            if not df.empty:  # If the CSV file is not empty
                # Extract the 'stand' value from the file name
                csv_for_result.append(files)
                filename = os.path.basename(files)  # Get the file name without the path
                stand = filename.split('_')[-1].split('.')[0]  # Extract the 'stand' part
                stand_id_result.append(stand)  # Add the 'stand' to the list
        except pd.errors.EmptyDataError:
            # Handle the case where the CSV file is empty or invalid
            print(f"{file} is empty or has no valid data.")
    return csv_for_result, stand_id_result
    



In [9]:

def prediction(csv_for_result):
    # Compile and run Java code with subprocess
    cwd = os.getcwd()  # Assuming cwd is defined earlier
    JAVA_SERVICE_PATH = os.path.join(cwd, 'source/java')
    H2O_GENMODEL_JAR_PATH = os.path.join(JAVA_SERVICE_PATH, 'h2o-genmodel.jar')
    GSON_JAR_PATH = os.path.join(JAVA_SERVICE_PATH, 'gson-2.8.8.jar')
    MAIN_JAVA_FILE_PATH = os.path.join(JAVA_SERVICE_PATH, 'main1.java')

    # Check if Java files and JARs exist
    if not os.path.exists(H2O_GENMODEL_JAR_PATH):
        raise FileNotFoundError(f"JAR file not found: {H2O_GENMODEL_JAR_PATH}")
    if not os.path.exists(GSON_JAR_PATH):
        raise FileNotFoundError(f"JAR file not found: {GSON_JAR_PATH}")
    if not os.path.exists(MAIN_JAVA_FILE_PATH):
        raise FileNotFoundError(f"Java file not found: {MAIN_JAVA_FILE_PATH}")

    # Use the correct classpath separator
    classpath_separator = os.pathsep
    classpath = f'.{classpath_separator}{H2O_GENMODEL_JAR_PATH}{classpath_separator}{GSON_JAR_PATH}'

    predictions_and_stats_list = []

    try:
        for item in csv_for_result:
            compile_command = ['javac', '-cp', classpath, 'main1.java']
            subprocess.run(compile_command, cwd=JAVA_SERVICE_PATH, check=True)
            # Execute the Java code and capture the output
            run_command = ['java', '-cp', classpath, 'main1', item]
            result = subprocess.run(run_command, cwd=JAVA_SERVICE_PATH, capture_output=True, text=True)
            if result.returncode != 0:
                raise RuntimeError(f"Java execution failed with code {result.returncode}")

            # Process the output (assuming it's JSON)
            predictions_and_stats = json.loads(result.stdout)
            # Round statistical metrics to the nearest integer
            rounded_predictions = {metric: round(value) for metric, value in predictions_and_stats.items()}
            predictions_and_stats_list.append(rounded_predictions)
    except subprocess.CalledProcessError as e:
        print(f"Error executing Java code: {e}")
        raise RuntimeError('Error executing Java code') from e  # Handle error as needed
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON output from Java: {e}")
        raise RuntimeError('Error decoding JSON output from Java') from e
    return predictions_and_stats_list

In [10]:
# Folder path
folder_path = os.path.join(os.getcwd(), "source/csv_files")
csv_for_result, stand_id_result = filter_csv(folder_path)

print(csv_for_result, stand_id_result)
predictions_and_stats_list = prediction(csv_for_result)
df = pd.DataFrame(predictions_and_stats_list)
df['StandId'] = [items for items in stand_id_result]

# Move 'StandId' to the first column
cols = ['StandId'] + [col for col in df.columns if col != 'StandId']
df = df[cols]


['d:\\trails\\kalamath_model_jupyter_package_testing\\source/csv_files\\csv_for_stand_101.csv', 'd:\\trails\\kalamath_model_jupyter_package_testing\\source/csv_files\\csv_for_stand_102.csv', 'd:\\trails\\kalamath_model_jupyter_package_testing\\source/csv_files\\csv_for_stand_103.csv', 'd:\\trails\\kalamath_model_jupyter_package_testing\\source/csv_files\\csv_for_stand_104.csv', 'd:\\trails\\kalamath_model_jupyter_package_testing\\source/csv_files\\csv_for_stand_105.csv', 'd:\\trails\\kalamath_model_jupyter_package_testing\\source/csv_files\\csv_for_stand_106.csv', 'd:\\trails\\kalamath_model_jupyter_package_testing\\source/csv_files\\csv_for_stand_107.csv'] ['101', '102', '103', '104', '105', '106', '107']


In [11]:
df

,StandId,mode,median,mean,highestValue,lowestValue
0,101,634,625,614,703,429
1,102,532,481,483,668,308
2,103,685,626,610,719,428
3,104,571,571,571,674,365
4,105,508,515,548,807,420
5,106,497,483,469,695,305
6,107,616,646,644,854,422


In [12]:
# Folder path
folder_path = modified_csv_folder
csv_for_result, stand_id_result = filter_csv(folder_path)
# print(csv_for_result)
df2=None
if csv_for_result != [] and stand_id_result != []:
    predictions_and_stats_list = prediction(csv_for_result)
    df2 = pd.DataFrame(predictions_and_stats_list)
    df2['StandId'] = [items for items in stand_id_result]
    df2['Curr_QMD'] = [int(values[0]) for item, values in dict_for_standIDnCrrnQMD_from_allFields.items()]
    df2['Curr_TPA'] = [int(values[1]) for item, values in dict_for_standIDnCrrnQMD_from_allFields.items()]
    # Move 'StandId' to the first column
    cols = ['StandId'] + [col for col in df2.columns if col != 'StandId']
    df2 = df2[['StandId','median', 'Curr_QMD','Curr_TPA' ]]
    df2 = df2.rename(columns={'median': 'Curr_SDImax'})
    df2['%SDImax'] =  (df2['Curr_TPA'] * 100 )/ df2['Curr_SDImax']
    display("stats with current TPA and current SDImax")
    display(df2)

    

'stats with current TPA and current SDImax'

,StandId,Curr_SDImax,Curr_QMD,Curr_TPA,%SDImax
0,101,950,8,700,73.684211
1,102,394,11,400,101.522843
2,103,482,12,390,80.912863


In [13]:
if df2 is not None:
    merged_df = pd.merge(df, df2, how='outer', on='StandId')
    merged_df = merged_df.rename(columns={'Curr_SDImax_x': 'median', 'Curr_SDImax_y': 'Curr_SDImax'})
    display(merged_df)  
    result_folder = os.path.join(cwd,'result/result_with_current_QMD_and_TPA.csv')
    df.to_csv(result_folder)


,StandId,mode,median,mean,highestValue,lowestValue,Curr_SDImax,Curr_QMD,Curr_TPA,%SDImax
0,101,634,625,614,703,429,950.0,8.0,700.0,73.684211
1,102,532,481,483,668,308,394.0,11.0,400.0,101.522843
2,103,685,626,610,719,428,482.0,12.0,390.0,80.912863
3,104,571,571,571,674,365,NaN,NaN,NaN,NaN
4,105,508,515,548,807,420,NaN,NaN,NaN,NaN
5,106,497,483,469,695,305,NaN,NaN,NaN,NaN
6,107,616,646,644,854,422,NaN,NaN,NaN,NaN


In [14]:
result_folder = os.path.join(cwd,'result/result.csv')
df.to_csv(result_folder)